In [40]:
from deap import creator, base, tools
from multiprocessing import Pool
from function_opt.functions import rastrigin
from function_opt.ga_scheme import eaMuPlusLambda
from typing import Callable, Optional, Tuple
import numpy as np

In [12]:
creator.create("BaseFitness", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.BaseFitness)

In [38]:
def mutation(individual):
    n = len(individual)
    for i in range(n):
        if rnd.random() < n * 0.15:
            individual[i] += rnd.normal(0.0, 0.2)
            individual[i] = np.clip(individual[i], -5, 5)
    return individual,

class Experiment:

    def _factory(self):
        return np.random.random(self.dim) * 10 - 5
    
    def __init__(self, 
                 func: Callable[[np.array], np.array], 
                 dim: int,
                 population: int,
                 max_iter: int,
                 mut_prob: float,
                 cross_prob: float,
                 pool: Pool = Pool(5),
                 engine: base.Toolbox = base.Toolbox(),
                 factory: Optional[Callable[[], np.array]] = None):
        self.population: int = population
        self.max_iter: int = max_iter
        self.mut_prob: float = mut_prob
        self.cross_prob: float = cross_prob
        self.func: Callable[float, float] = func
        self.dim: int = dim
        self.pool: Pool = pool
        self.engine: base.Toolbox = engine
        self.factory: Callable[[], np.array] = factory
        if self.factory is None:
            self.factory = self._factory

        self.engine.register("map", self.pool.map)
        
        self.engine.register("individual", 
            tools.initIterate, 
            creator.Individual, 
            self.factory
        )

        self.engine.register("population", 
            tools.initRepeat,
            list,
            self.engine.individual,
            self.population
        )

        self.engine.register("mate", tools.cxTwoPoint)
    
        self.engine.register("mutate", 
            tools.mutUniformInt, 
            low=-1, 
            up=1, 
            indpb=0.1)

        self.engine.register("select", tools.selTournament, tournsize=3)
        self.engine.register("evaluate", self.func)

    def run(self):
        pop: int = self.engine.population()
        hof: tools.HallOfFame = tools.HallOfFame(3, np.array_equal)
        stats: tools.Statistics = tools.Statistics(lambda x: x.fitness.values[0])
        
        stats.register("avg", np.mean)
        stats.register("std", np.std)
        stats.register("min", np.min)
        stats.register("max", np.max)

        pop, log = eaMuPlusLambda(
            pop, 
            self.engine,
            mu=self.population,
            lambda_= int(self.population * 0.8),
            cxpb=self.cross_prob,
            mutpb=self.mut_prob,
            ngen=self.max_iter,
            stats=stats,
            halloffame=hof,
            verbose=True
        )

        print("Best = {}".format(hof[0]))
        print("Best fit = {}".format(hof[0].fitness.values[0]))
        return log

In [39]:
def obj(x: np.array) -> Tuple[np.array]:
    return (rastrigin(x),)

dim = 100
population_size = 100
max_iter = 1000
num_iter = 0
scene = Experiment(
    func=obj,
    dim=dim,
    population=population_size,
    max_iter=1000,
    mut_prob=0.4,
    cross_prob=0.6
)

log = scene.run()

PicklingError: Can't pickle <function <lambda> at 0x7ffd2dca5950>: attribute lookup <lambda> on __main__ failed

In [ ]:
from function_opt.draw_log import draw_log
draw_log(log)